In [2]:
import json
import pandas as pd
import requests
from datetime import date

In [3]:
api_key = "96b6cae048b16e876ac91564d0834d2e"
#We are using the TMDb API to collect revenue data on movies

In [4]:
#build the url to get movies sorted by there revenue data
url = 'https://api.themoviedb.org/3/discover/movie?api_key=' +  api_key + '&sort_by=revenue.desc'

response = requests.get(url).json()

#From the results grab the results section which actually contains the data
film_revenue = response["results"]

#We are looping through this number of pages to get the data for a total of over 1000 movies, since that is how many are in our dataset
for page in range(2, 55):
    try:
        #Get results for each page and add to existing results
        response = requests.get(url + f"&page={page}").json()
        film_revenue.extend(response["results"])
    except:
        pass



In [5]:
#Create the dataframe to store the results of our API queries
columns = ['film', 'revenue']
df = pd.DataFrame(columns=columns)

In [6]:
#For each movie we are looking into...
for film in film_revenue:
    #Get the actual data for the film and convert to json
    film_revenue = requests.get('https://api.themoviedb.org/3/movie/'+ str(film['id']) +'?api_key='+ api_key+'&language=en-US')
    film_revenue = film_revenue.json()
    df.loc[len(df)]=[film['title'],film_revenue['revenue']] # store title and revenue in our dataframe    

In [9]:
df['film']=df['film'].str.replace(',','') # clean up the data 

#Create an IDs column so that when we load this dataframe into SQL as a table, we have matching IDs between this table and our inventory table.
#Having matching IDs will make it very easy to join this data into our inventory table
ids = []
for i in range(1080):
    ids.append(i+1)
df['id'] = ids
print(df)

df.to_csv('apidat.csv')

                                                   film     revenue    id
0                                        Watch Your Six  4999999999     1
1                                                Avatar  2920357254     2
2                                     Avengers: Endgame  2799439100     3
3                                             Condemned  2470000000     4
4                              Avatar: The Way of Water  2309660236     5
...                                                 ...         ...   ...
1075                                  The Scorpion King   165333180  1076
1076                                        The Muppets   165184237  1077
1077                                  Wonder Woman 1984   165160005  1078
1078  Michael Jackson: HIStory World Tour - Live in ...   165000000  1079
1079     Michael Jackson: HIStory Tour - Live in Munich   165000000  1080

[1080 rows x 3 columns]
